<a href="https://colab.research.google.com/github/mkprasadp/NLP/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Task 1

In [ ]:
import re
pattern1 = re.compile(r'abc')
result1 = pattern1.match('abcdef')
if result1:
  print("Match found:", result1.group())

pattern2 = re.compile(r'.')
result2 = pattern2.search('Hello')
if result2:
  print("Character found:", result2.group())

pattern3 = re.compile(r'[aeiou]')
result3 = pattern3.search('Hello')
if result3:
  print("Vowel found:", result3.group())

pattern4 = re.compile(r'\d{3}-\d{2}-\d{4}')
result4 = pattern4.match('123-45-6789')
if result4:
  print("Social Security Number:", result4.group())

pattern5 = re.compile(r'\.')
result5 = pattern5.search('www.example.com')
if result5:
  print("Dot found:", result5.group())

pattern6 = re.compile(r'(\d{2})/(\d{2})/(\d{4})')
result6 = pattern6.match('23/06/2025')
if result6:
  print("Day:", result6.group(1))
  print("Month:", result6.group(2))
  print("Year:", result6.group(3))

pattern7 = re.compile(r'cat|dog')
result7 = pattern7.search('I have a cat and a dog')
if result7:
  print("Animal found:", result7.group())

Match found: abc
Character found: H
Vowel found: e
Social Security Number: 123-45-6789
Dot found: .
Day: 23
Month: 06
Year: 2025
Animal found: cat


#Task 2

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

text = "Tokenization without trnsformer is Straightforward with tools like NLTK"
tokens = word_tokenize(text)
print("Tokens:",tokens);

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
text = "Tokenization without transformers is Straightforward with tools like NLTK"
tokens_transformers = tokenizer(text,return_tensors="pt")
print("Tokens Transformer:",tokens_transformers);

tokens_transformers_list = tokenizer.convert_ids_to_tokens(tokens_transformers["input_ids"][0].numpy().tolist());
print("Transformers token(list):",tokens_transformers_list);

decode_text = tokenizer.decode(tokens_transformers['input_ids'][0],skip_special_tokens=True)
print("Decode Text:", decode_text);

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Tokens: ['Tokenization', 'without', 'trnsformer', 'is', 'Straightforward', 'with', 'tools', 'like', 'NLTK']
Tokens Transformer: {'input_ids': tensor([[  101, 19204,  3989,  2302, 19081,  2003, 19647,  2007,  5906,  2066,
         17953,  2102,  2243,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
Transformers token(list): ['[CLS]', 'token', '##ization', 'without', 'transformers', 'is', 'straightforward', 'with', 'tools', 'like', 'nl', '##t', '##k', '[SEP]']
Decode Text: tokenization without transformers is straightforward with tools like nltk


#Task 3

In [ ]:
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
import spacy
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

nltk.download('punkt')
nltk.download('punkt_tab')
spacy.cli.download("en_core_web_sm")
nlp = spacy.load("en_core_web_sm")

corpus = "One disadvantage of using 'Best Of' samping is that it may lead to limited exploration of the model's knowledge and creativity. By focusing on the most probable next words, the model might generate responses that are safe and conventional, potentially missing out on more diverse and innovative outputs. The lack of exploration could result in repetitive or less imaginative responses, especially in situations where novel and unconventional ideas are desired.To address this limitation, other sampling strategies like temperature-based sampling or top-p (nucleus) sampling can be employed to introduce more randomness and encourage the model to explore a broader range of possibilities. However, it's essential to carefully balance exploration and exploitation based on the specific requirements of the task or application."

tokens = word_tokenize(corpus)
lemmatized_tokens = [token.lemma_ for token in nlp(corpus)]
all_tokens = tokens + lemmatized_tokens

tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_tokens)
total_words = len(tokenizer.word_index) + 1

input_sequences = []
for line in all_tokens:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

max_sequence_length = max(len(seq) for seq in input_sequences)
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_length, padding='pre')

X, y = input_sequences[:, :-1], input_sequences[:, -1]
y = np.array(y)

model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_length-1))
model.add(LSTM(100))
model.add(Dense(total_words, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X, y, epochs=10, verbose=1)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step - accuracy: 0.0000e+00 - loss: 4.6768
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step - accuracy: 0.0000e+00 - loss: 4.6689
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step - accuracy: 0.6667 - loss: 4.6610
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - accuracy: 1.0000 - loss: 4.6530
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - accuracy: 1.0000 - loss: 4.6450
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step - accuracy: 1.0000 - loss: 4.6369
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step - accuracy: 1.0000 - loss: 4.6286
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 1.0000 - los

#Task 4

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

def load_document(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

def tokenize_document(document):
    tokens = word_tokenize(document)
    return [word.lower() for word in tokens if word.isalpha()]

def remove_stopwords(tokens):
    stop_words = set(stopwords.words('english'))
    return [word for word in tokens if word not in stop_words]

def find_morphology(tokens):
    fdist = FreqDist(tokens)
    return fdist.most_common(10)

document_path = '/content/tourism_documents.txt'

document = load_document(document_path)
tokens = tokenize_document(document)
tokens_without_stopwords = remove_stopwords(tokens)
morphology = find_morphology(tokens_without_stopwords)

print("Morphology of the document:")
for word, frequency in morphology:
    print(f"{word}: {frequency}")

Morphology of the document:
traveling: 1
abroad: 1
make: 1
sure: 1
prepare: 1
necessary: 1
including: 1
passport: 1
visa: 1
travel: 1


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


#Task 5

In [ ]:
import string
import random
import nltk
from nltk.corpus import stopwords, reuters
from collections import Counter, defaultdict
from nltk import FreqDist, ngrams
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('reuters')
sents = reuters.sents()
stop_word = set(stopwords.words('english'))
string.punctuation = string.punctuation + ' " ' + ' " ' + ' - ' + ' _ '
removal_list = list(stop_word) + list(string.punctuation) + ['\t', 'rt']
unigram = []
bigram = []
trigram = []
tokenized_text = []
for sentence in sents:
    sentence = list(map(lambda x: x.lower(), sentence))
    for word in sentence:
        if word == '.':
            sentence.remove(word)
        else:
            unigram.append(word)
            tokenized_text.append(word)
    bigram.extend(list(ngrams(sentence, 2, pad_left=True, pad_right=True)))
    trigram.extend(list(ngrams(sentence, 3, pad_left=True, pad_right=True)))
def remove_stopwords(x):
    y = []
    for item in x:
        if isinstance(item, tuple):
            count = 0
            for word in item:
                if word not in removal_list:
                    count = 1
                    break
            if count == 1:
                y.append(item)
        else:
            if item not in removal_list:
                y.append(item)
    return y

unigram = remove_stopwords(unigram)
bigram = remove_stopwords(bigram)
trigram = remove_stopwords(trigram)
freq_uni = FreqDist(unigram)
freq_bi = FreqDist(bigram)
freq_tri = FreqDist(trigram)
d = defaultdict(Counter)
for a, b, c in freq_tri:
    if (a is not None) and (b is not None) and (c is not None):
        d[a, b][c] += freq_tri[a, b, c]
def pick_word(counter):
    "choose a random element"
    return random.choice(list(counter.elements()))
prefix = "he", "is"
print(" ".join(prefix))
s = " ".join(prefix)
for i in range(19):
    if prefix in d:
        suffix = pick_word(d[prefix])
        s = s + ' ' + suffix
        print(s)
        prefix = prefix[1], suffix
    else:
        print(f"Prefix {prefix} not found in dictionary. Stopping text generation.")
        break

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package reuters to /root/nltk_data...
[nltk_data]   Package reuters is already up-to-date!


he is
he is still
he is still just
he is still just a
he is still just a question
he is still just a question from
he is still just a question from the
he is still just a question from the companies
he is still just a question from the companies '
he is still just a question from the companies ' price
he is still just a question from the companies ' price for
he is still just a question from the companies ' price for japanese
he is still just a question from the companies ' price for japanese makers
he is still just a question from the companies ' price for japanese makers under
he is still just a question from the companies ' price for japanese makers under the
he is still just a question from the companies ' price for japanese makers under the zero
he is still just a question from the companies ' price for japanese makers under the zero duty
he is still just a question from the companies ' price for japanese makers under the zero duty bindings
he is still just a question from the com

#Task 6

In [ ]:
from nltk.util import ngrams
from nltk.lm import Laplace
from nltk.tokenize import word_tokenize
from nltk.lm.preprocessing import padded_everygram_pipeline
import nltk

nltk.download('punkt_tab')

def ngram_smoothing(sentence, n):
  tokens = word_tokenize(sentence.lower())
  train_data, padded_sents = padded_everygram_pipeline(n, tokens)
  model = Laplace(n)
  model.fit(train_data, padded_sents)
  return model

sentence = "his is the sample example taken to evaluate the n-gram smoothening technique"
print("Enter a sentence: ",sentence)
n = 10
print("Enter the value of N for N-grams:",n)
model = ngram_smoothing(sentence, n)
context = tuple(sentence.lower().split()[-n+1:])
next_words = model.generate(3, text_seed=context)
print("Next words:", ' '.join(next_words))

Enter a sentence:  his is the sample example taken to evaluate the n-gram smoothening technique
Enter the value of N for N-grams: 10
Next words: </s> </s> </s>


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


#Task 7

In [ ]:
import nltk
nltk.download("all");
from nltk.tag import HiddenMarkovModelTagger
from nltk.corpus import treebank
nltk.download('treebank')
corpus = treebank.tagged_sents()
train_data = corpus[:3000]
test_data = corpus[3000:]
hmm_tagger = HiddenMarkovModelTagger.train(train_data)
sentance = "I Love NLP"
tokens = nltk.word_tokenize(sentance)
tagged_sentance = hmm_tagger.tag(tokens)
print(tagged_sentance)

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_r

[('I', 'PRP'), ('Love', 'MD'), ('NLP', 'VB')]


#Task 8

In [ ]:
!pip install -U nltk

In [20]:
import nltk
from nltk.corpus import treebank
from nltk.tag import hmm
from nltk.classify import MaxentClassifier
nltk.download('maxent_treebank_pos_tagger')
from nltk.tag import PerceptronTagger, StanfordTagger
nltk.download('treebank')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('averaged_perceptron_tagger')
corpus = list(treebank.tagged_sents())
train_data = corpus[:int(0.8 * len(corpus))]
test_data = corpus[int(0.8 * len(corpus)):]
hmm_tagger = hmm.HiddenMarkovModelTrainer().train(train_data)
hmm_accuracy = hmm_tagger.evaluate(test_data)
print(f"HMM Tagger Accuracy: {hmm_accuracy:.4f}")

[nltk_data] Downloading package maxent_treebank_pos_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/maxent_treebank_pos_tagger.zip.
[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
/tmp/ipython-input-3128377115.py:15: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  hmm_accuracy = hmm_tagger.evaluate(test_data)
/usr/local/lib/python3.12/dist-packages/nltk/tag/hmm.py:333: RuntimeWarning: overflow encountered in cast
  X[i, j] = self._transitions[si].lo

HMM Tagger Accuracy: 0.3647


#Task 9

In [ ]:
import nltk
import numpy as np
from sklearn.metrics import accuracy_score
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')
def hmm_pos_tagger(sentence):
  tokens = nltk.word_tokenize(sentence)
  tagged = nltk.pos_tag(tokens)
  return tagged
def log_linear_pos_tagger(sentence):
  tagged = nltk.pos_tag(nltk.word_tokenize(sentence))
  return tagged
def compare_performance(sentence):
  hmm_tags = hmm_pos_tagger(sentence)
  log_linear_tags = log_linear_pos_tagger(sentence)
  gold_standard_tags = [tag for _, tag in hmm_tags]
  hmm_predicted_tags = [tag for _, tag in hmm_tags]
  log_linear_predicted_tags = [tag for _, tag in log_linear_tags]
  hmm_accuracy = accuracy_score(gold_standard_tags, hmm_predicted_tags)
  log_linear_accuracy = accuracy_score(gold_standard_tags, log_linear_predicted_tags)

  print("HMM Predicted Tags:", hmm_predicted_tags)
  print("Log-Linear Model Predicted Tags:", log_linear_predicted_tags)
  print("HMM Accuracy:", hmm_accuracy)
  print("Log-Linear Model Accuracy:", log_linear_accuracy)

input_text = "The quick brown fox jumps over the lazy dog."
compare_performance(input_text)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


HMM Predicted Tags: ['DT', 'JJ', 'NN', 'NN', 'VBZ', 'IN', 'DT', 'JJ', 'NN', '.']
Log-Linear Model Predicted Tags: ['DT', 'JJ', 'NN', 'NN', 'VBZ', 'IN', 'DT', 'JJ', 'NN', '.']
HMM Accuracy: 1.0
Log-Linear Model Accuracy: 1.0


#Task 10

In [ ]:
from bs4 import BeautifulSoup
import spacy
nlp = spacy.load("en_core_web_sm")
def pos_tag_and_extract_info(text):
  doc = nlp(text)
  nouns = []
  verbs = []
  adjectives = []
  entities = []
  for token in doc:
    if token.pos_ == "NOUN":
      nouns.append(token.text)
    elif token.pos_ == "VERB":
      verbs.append(token.text)
    elif token.pos_ == "ADJ":
      adjectives.append(token.text)
    for entity in doc.ents:
      entities.append((entity.text, entity.label_))
  return nouns, verbs, adjectives, entities

web_document = """
<html>
<head>
<title>Example Web Page</title>
</head>
<body>
<p>This is an example web page. It contains some text with various parts of speech.</p>
<p>For example, "The cat jumps over the lazy dog" contains a noun, a verb, and prepositions.</p>
</body>
</html>
"""
def extract_text_from_html(html):
  soup = BeautifulSoup(html, 'html.parser')
  return soup.get_text()

text_content = extract_text_from_html(web_document)
nouns, verbs, adjectives, entities = pos_tag_and_extract_info(text_content)

print("Nouns:", nouns)
print("Verbs:", verbs)
print("Adjectives:", adjectives)
print("Entities:", entities)

Nouns: ['Web', 'example', 'web', 'page', 'text', 'parts', 'speech', 'example', 'cat', 'dog', 'verb', 'prepositions']
Verbs: ['contains', 'jumps', 'contains']
Adjectives: ['various', 'lazy']
Entities: []


#Task 11

In [19]:
import re, requests, torch
import torch.nn as nn
from bs4 import BeautifulSoup
import from tensorflow.keras.utils
from tensorflow.keras.utils import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
class ChunkerModel(nn.Module):
def __init__(self, vocab_size):
super().__init__()
self.lstm, self.fc = nn.LSTM(128, 64, batch_first=True), nn.Linear(64, 1)
def forward(self, x): return torch.sigmoid(self.fc(self.lstm(x)[0][:, -1, :]))
def fetch_text(url):
return re.sub(r'\s+', ' ', ' '.join(p.get_text() for p in BeautifulSoup(requests.get(url).text,
'html.parser').find_all('p')))
def preprocess(text):
tok = Tokenizer(num_words=5000)
tok.fit_on_texts([text])
return pad_sequences(tok.texts_to_sequences([text]), maxlen=100, padding='post'), tok
def segment_text(url):
text, (seq, tok) = fetch_text(url), preprocess(fetch_text(url))
return text.split('. ')[:5]
print("Extracted Chunks:",
*segment_text("https://en.wikipedia.org/wiki/Natural_language_processing"), sep="\n")

SyntaxError: invalid syntax (ipython-input-634716896.py, line 4)